# The Fitting

When the prediction is done, there is a density parameter grid, and the element abundances relating to all the parameters to the density parameter grid, for each spectrum. In this notebook, we will use the observed supernova spectral time sequence, to determine the best density parameter among the grid. Using that best density parameter and the relating element abundance and luminosity and photosphere velocity, all the TARDIS simulated spectra should be consistent to the observed spectra.  

Because all the spectra from a supernova should share a same density structure, type Ia supernova ejecta expands homologusly around this time. 

# The Fitting Code

You can run the "GridUltraSearch.py" code to fit the model onto a spectral time sequence.  
When running this code, you will need to change several things. Firstly, you will need to find a cache directory, which stores the input file and spectra of TARDIS simulation, and an output directory, which stores the simulated spectral time sequence. Secondly, you will need to change "tamulauncher" to something else, it is a load-balance program only on Texas A&M University supercomputers and you probably don't have that thing. That program is very helpful when running multiple TARDIS simulations on multiple machines. Thirdly, you will specify which element data base you want to use for TARDIS simulation.  


In [5]:
!python GridUltraSearch.py SN2011fe

# Extract the Best

In the folder "predOutMassive", there are already many simulation spectra of the supernovae used in the paper. To extract the best density profile for a certain supernova, which can fit most of the observed spectra, here it is the code. 

In [2]:
import os
import sys
import time
import glob
import yaml
#import tardis
import astropy
import numpy as np
import pandas as pd
import astropy.units as u
import astropy.constants as c
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.ndimage.filters import gaussian_filter1d

# Something Boring

Similar to what you have done in the "GridUltraSearch.py" script, you will need to specify several paths.  
Firstly, all the spectra should be stored in "predOutMassive".  
Secondly, your supernova list should be in "dm15List2.csv". This list is similar to the "dm15WithDens.csv" list, which is given in the folder, but no need to include the density parameters "pivot" and "slope", as we are going to find them soon.  
Thirdly, change the "homedir", "cachedir", and "specoutdir" as is in the "GridUltraSearch.py" script.  

In [ ]:
snNameList=[i.split('/')[-1] for i in glob.glob('predOutMassive/*')]
dm15List=pd.read_csv('dm15List2.csv')

In [3]:
solarLumi=3.846*10**33*u.erg/u.s
wave=np.genfromtxt('Prim.ascii')[:,0]
def Normalizer(spec,shortwave=6500,longwave=7500):
    small=np.argmin(abs(spec[:,0]-shortwave))
    long=np.argmin(abs(spec[:,0]-longwave))
    if small<long:spec[:,1]=spec[:,1]/np.average(spec[small:long,1])
    if small>long:spec[:,1]=spec[:,1]/np.average(spec[long:small,1])
    return spec


In [ ]:
homedir='/scratch/user/chenxingzhuo/DLTD/NewNewIGE/'
cachedir='/scratch/user/chenxingzhuo/DLTD/SuperLargeFit/CacheTest/'
specoutdir='/scratch/user/chenxingzhuo/DLTD/SuperLargeFitRefine/SpecOutMassive/'

In [ ]:
IGEdens=np.genfromtxt(homedir+'IGenhance/Density.dat',skip_header=1)
IGEelem=np.genfromtxt(homedir+'IGenhance/IGenhanceElem.dat')
IGEvelo=IGEdens[:,1]
IGEelem[:,1:]=IGEelem[:,1:]-10**-6+10**-8

In [ ]:
def BestSelecter(FitFluxIn,ObsSpecIn,matBigIn,auxBigIn,returnList=False):
    pivotList=[]
    pivotIndexList=[]
    slopeList=[]
    slopeIndexList=[]
    chisqList=[]
    for pivInd in range(FitFluxIn.shape[0]):
        for sloInd in range(FitFluxIn.shape[1]):
            fluxList=FitFluxIn[pivInd,sloInd]
            if np.isnan(np.sum(fluxList)):continue
            pivot=auxBigIn[pivInd,sloInd,0,3]
            slope=auxBigIn[pivInd,sloInd,0,4]
            pivotList.append(round(float(pivot),3))
            slopeList.append(round(float(slope),3))
            pivotIndexList.append(pivInd)
            slopeIndexList.append(sloInd)
            chisqOneSeq=0
            for j in range(len(ObsSpecIn)):
                fluxObs=ObsSpecIn[j][:,0]
                fluxFit=fluxList[j]
                mask=(fluxObs>0)
                fluxObs=fluxObs[mask]
                fluxFit=fluxFit[mask]
                fluxObs=fluxObs/np.mean(fluxObs)
                fluxFit=fluxFit/np.mean(fluxFit)
                chisqOneSeq=chisqOneSeq+np.sum((fluxObs-fluxFit)**2)
            chisqList.append(chisqOneSeq)
    chisqList=np.array(chisqList)
    print(chisqList)
    print(pivotList)
    print(slopeList)
    minHere=np.nanargmin(chisqList)
    pivotCen=pivotList[minHere]
    slopeCen=slopeList[minHere]
    if returnList==False:
        return pivotCen,slopeCen
    else:
        return pivotCen,slopeCen,chisqList,pivotList,pivotIndexList,slopeList,slopeIndexList

# The Best! 

The simulated spectral sequence with the best density parameter will be stored in the folder "BestSeq", and the list "dm15List" will add two columns of the density parameters.  

In [ ]:
dm15List['pivot']=np.nan
dm15List['slope']=np.nan
for i in range(len(dm15List)):
    snname=dm15List['ObjName'].iloc[i]
    
    try:FitFlux=np.load(specoutdir+snname+'_FitFlux.npy')
    except:continue
    
    matBig=np.load('predOutMassive/'+snname+'/matBig.npy')
    errBig=np.load('predOutMassive/'+snname+'/errBig.npy')
    auxBig=np.load('predOutMassive/'+snname+'/auxBig.npy')
    aerBig=np.load('predOutMassive/'+snname+'/aerBig.npy')
    Xinput=np.load('predOutMassive/'+snname+'/Xinput.npy')
    try:pivotCen,slopeCen,chisqList,pivotList,pivotIndexList,slopeList,slopeIndexList=BestSelecter(FitFlux,Xinput,matBig,auxBig,returnList=True)
    except:continue
    dm15List['pivot'].iloc[i]=pivotCen
    dm15List['slope'].iloc[i]=slopeCen
    
    bestSeq=FitFlux[pivotIndexList[np.nanargmin(chisqList)],slopeIndexList[np.nanargmin(chisqList)]]
    np.save('BestSeq/'+snname+'.npy',bestSeq)

In [ ]:
dm15List.to_csv('dm15WithDens.csv')